In [1]:
# %%capture
!pip install -r requirements.txt

In [80]:
import os
import openai
from string import Template
import json
from neo4j import GraphDatabase
import glob
from timeit import default_timer as timer
from dotenv import load_dotenv
from time import sleep

In [81]:
# Load environment variables
load_dotenv('.env')

True

In [82]:
# OpenAI API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.api_version = os.getenv("OPENAI_API_VERSION")
openai_deployment = "gpt-4"
# print(openai.api_key)

In [87]:
# Neo4j configuration & constraints
neo4j_url = os.getenv("NEO4J_CONNECTION_URL")
neo4j_user = os.getenv("NEO4J_USER")
neo4j_password = os.getenv("NEO4J_PASSWORD")
gds = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))

### 2. Helper Functions

In [100]:
# Function to call the OpenAI API
def process_gpt(file_prompt, system_msg):
    client = openai.OpenAI()  # Create a client instance
    completion = client.chat.completions.create(  # Use the new API structure
        model=openai_deployment,  # Update this to the correct model name, e.g., "gpt-4"
        max_tokens=1000,
        temperature=0,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": file_prompt},
        ],
    )
    nlp_results = completion.choices[0].message.content
    sleep(8)
    return nlp_results


# def extract_entities_relationships(folder, prompt_template):
#     start = timer()
#     files = glob.glob(f"./data/{folder}/*")
#     system_msg = "You are a beauty and personal care expert who understands product relationships and routines."
#     print(f"Running pipeline for {len(files)} files in {folder} folder")
    
#     # Maintain context across products
#     product_context = {
#         'brands': set(),
#         'price_tiers': {
#             'budget': [],
#             'mid_range': [],
#             'premium': []
#         },
#         'rating_tiers': {
#             'excellent': [],
#             'good': [],
#             'average': [],
#             'poor': []
#         },
#         'categories': {},
#         'concerns': {},
#         'ingredients': set()
#     }
    
#     all_entities = {}
#     all_relationships = set()
    
#     try:
#         # First pass: collect product information
#         for file in files:
#             try:
#                 with open(file, "r", encoding='utf-8') as f:
#                     text = f.read().rstrip()
#                     prompt = Template(prompt_template).substitute(ctext=text)
#                     result = process_gpt(prompt, system_msg=system_msg)
#                     parsed_result = json.loads(result)
                    
#                     # Update product context
#                     for entity in parsed_result["entities"]:
#                         if entity["label"] == "Product":
#                             # Categorize by price tier
#                             price = entity.get("price", 0)
#                             if price < 15:
#                                 product_context['price_tiers']['budget'].append(entity["id"])
#                             elif price < 30:
#                                 product_context['price_tiers']['mid_range'].append(entity["id"])
#                             else:
#                                 product_context['price_tiers']['premium'].append(entity["id"])
                                
#                             # Categorize by rating tier
#                             rating = entity.get("rating", 0)
#                             if rating >= 4.5:
#                                 product_context['rating_tiers']['excellent'].append(entity["id"])
#                             elif rating >= 4.0:
#                                 product_context['rating_tiers']['good'].append(entity["id"])
#                             elif rating >= 3.0:
#                                 product_context['rating_tiers']['average'].append(entity["id"])
#                             else:
#                                 product_context['rating_tiers']['poor'].append(entity["id"])
                                
#                             # Track categories and brands
#                             if "category" in entity:
#                                 product_context['categories'].setdefault(entity["category"], []).append(entity["id"])
#                             if "brand" in entity:
#                                 product_context['brands'].add(entity["brand"])
                            
#             except Exception as e:
#                 print(f"Error in first pass processing {file}: {e}")
#                 continue

#         # Second pass: create relationships using context
#         for file in files:
#             try:
#                 with open(file, "r", encoding='utf-8') as f:
#                     text = f.read().rstrip()
#                     prompt = Template(prompt_template).substitute(ctext=text, context=str(product_context))
#                     result = process_gpt(prompt, system_msg=system_msg)
#                     parsed_result = json.loads(result)
                    
#                     # Add entities and relationships
#                     for entity in parsed_result["entities"]:
#                         entity_id = entity["id"]
#                         if entity_id in all_entities:
#                             all_entities[entity_id].update(entity)
#                         else:
#                             all_entities[entity_id] = entity
                    
#                     all_relationships.update(parsed_result["relationships"])
                    
#                     # Add contextual relationships
#                     current_product_id = next(e["id"] for e in parsed_result["entities"] if e["label"] == "Product")
                    
#                     # Add price tier relationships
#                     for product_id in product_context['price_tiers'][get_price_tier(current_product_id, all_entities)]:
#                         if product_id != current_product_id:
#                             all_relationships.add(f"{current_product_id}|SIMILAR_PRICE_TIER_AS|{product_id}")
                    
#                     # Add rating tier relationships
#                     for product_id in product_context['rating_tiers'][get_rating_tier(current_product_id, all_entities)]:
#                         if product_id != current_product_id:
#                             all_relationships.add(f"{current_product_id}|SIMILAR_RATING_AS|{product_id}")
                    
#                     # Add category relationships
#                     current_category = all_entities[current_product_id].get("category")
#                     if current_category and current_category in product_context['categories']:
#                         for product_id in product_context['categories'][current_category]:
#                             if product_id != current_product_id:
#                                 all_relationships.add(f"{current_product_id}|SAME_CATEGORY_AS|{product_id}")
                    
#             except Exception as e:
#                 print(f"Error in second pass processing {file}: {e}")
#                 continue
                
#     except Exception as e:
#         print(f"Error in main processing: {e}")
#         return []
        
#     final_result = {
#         "entities": list(all_entities.values()),
#         "relationships": list(all_relationships)
#     }
    
#     end = timer()
#     print(f"Pipeline completed in {end-start} seconds")
#     return [final_result]

# def get_price_tier(product_id, entities):
#     price = entities[product_id].get("price", 0)
#     if price < 15:
#         return "budget"
#     elif price < 30:
#         return "mid_range"
#     return "premium"

# def get_rating_tier(product_id, entities):
#     rating = entities[product_id].get("rating", 0)
#     if rating >= 4.5:
#         return "excellent"
#     elif rating >= 4.0:
#         return "good"
#     elif rating >= 3.0:
#         return "average"
#     return "poor"


def extract_entities_relationships(folder, prompt_template):
    start = timer()
    files = glob.glob(f"./data/{folder}/*")
    system_msg = "You are a beauty and personal care expert who understands product relationships and routines."
    print(f"Running pipeline for {len(files)} files in {folder} folder")
    
    # First pass: collect and summarize all product information
    product_summaries = []
    summary_prompt = """
    Create a concise summary of the product with key information in JSON format:
    {
        "id": "product_id",
        "name": "product name",
        "brand": "brand name",
        "category": "main category",
        "price": float,
        "rating": float,
        "key_features": ["list of 3-5 most important features or benefits"],
        "main_ingredients": ["list of key ingredients if mentioned"],
        "target_concerns": ["list of skin/hair concerns it addresses"],
        "usage_context": "when/how the product is meant to be used"
    }

    Extract only the most relevant information from the description that would help identify relationships with other products.

    Product text:
    $ctext
    """
    
    print("First pass: Collecting product summaries...")
    for file in files:
        try:
            with open(file, "r", encoding='utf-8') as f:
                text = f.read().rstrip()
                prompt = Template(summary_prompt).substitute(ctext=text)
                result = process_gpt(prompt, system_msg)
                summary = json.loads(result)
                summary['file_id'] = file.split("/")[-1].replace(".md", "")
                product_summaries.append(summary)
                print(f"Summarized {file.split('/')[-1]}")
        except Exception as e:
            print(f"Error summarizing {file}: {e}")
            continue
    
    # Create a structured context string with product summaries
    context = "Available products in the database:\n"
    for summary in product_summaries:
        context += f"""
        ID: {summary['file_id']}
        Name: {summary['name']}
        Brand: {summary['brand']}
        Category: {summary['category']}
        Price: {summary['price']} dollars
        Rating: {summary['rating']}
        Key Features: {', '.join(summary['key_features'])}
        Main Ingredients: {', '.join(summary.get('main_ingredients', ['Not specified']))}
        Target Concerns: {', '.join(summary.get('target_concerns', ['Not specified']))}
        Usage: {summary['usage_context']}
        ---
        """
    
    # Second pass: process each product with context
    all_entities = {}
    all_relationships = set()
    
    print("\nSecond pass: Processing products with context...")
    for file in files:
        try:
            with open(file, "r", encoding='utf-8') as f:
                text = f.read().rstrip()
                current_product_id = file.split("/")[-1].replace(".md", "")
                
                # Create a focused context with:
                # 1. The full details of the current product
                # 2. Summaries of products with similar characteristics
                current_summary = next(s for s in product_summaries if s['file_id'] == current_product_id)
                
                # Filter relevant products for focused context
                relevant_products = []
                for summary in product_summaries:
                    if summary['file_id'] == current_product_id:
                        continue
                    
                    # Check for relevance based on various factors
                    price_diff = abs(summary['price'] - current_summary['price'])
                    same_brand = summary['brand'].lower() == current_summary['brand'].lower()
                    same_category = summary['category'].lower() == current_summary['category'].lower()
                    
                    if (price_diff <= 10 or same_brand or same_category or 
                        any(concern in current_summary.get('target_concerns', []) 
                            for concern in summary.get('target_concerns', []))):
                        relevant_products.append(summary)
                
                # Create focused context
                focused_context = f"""
                Current Product Full Details:
                {text}

                Relevant Products for Relationship Consideration:
                """
                
                for prod in relevant_products[:10]:  # Limit to 10 most relevant products
                    focused_context += f"""
                    ID: {prod['file_id']}
                    Name: {prod['name']}
                    Brand: {prod['brand']}
                    Category: {prod['category']}
                    Price: {prod['price']} dollars
                    Rating: {prod['rating']}
                    Key Features: {', '.join(prod['key_features'])}
                    Main Ingredients: {', '.join(prod.get('main_ingredients', ['Not specified']))}
                    Target Concerns: {', '.join(prod.get('target_concerns', ['Not specified']))}
                    Usage: {prod['usage_context']}
                    ---
                    """
                
                # Add focused context to the prompt
                full_prompt = f"""
                {prompt_template}

                {focused_context}

                Important: Consider relationships with other available products based on:
                1. Similar price ranges (within 10 dollars)
                2. Same brand or manufacturer
                3. Complementary products in beauty routines
                4. Similar product categories or subcategories
                5. Common target concerns or purposes
                6. Compatible ingredients
                7. Similar usage contexts
                8. Routine sequence (products used before/after)
                """
                
                result = process_gpt(full_prompt, system_msg)
                parsed_result = json.loads(result)
                
                # Add entities and relationships
                for entity in parsed_result["entities"]:
                    entity_id = entity["id"]
                    if entity_id in all_entities:
                        all_entities[entity_id].update(entity)
                    else:
                        all_entities[entity_id] = entity
                
                all_relationships.update(parsed_result["relationships"])
                print(f"Processed {file.split('/')[-1]}")
                
        except Exception as e:
            print(f"Error processing {file}: {e}")
            continue
    
    final_result = {
        "entities": list(all_entities.values()),
        "relationships": list(all_relationships)
    }
    
    end = timer()
    print(f"Pipeline completed in {end-start} seconds")
    return [final_result]


# def clean_text(text):
#     """Clean text to prevent JSON parsing issues"""
#     # Replace problematic characters
#     text = text.replace('"', "'")  # Replace double quotes with single quotes
#     text = text.replace('\\', '/')  # Replace backslashes with forward slashes
#     text = text.encode('ascii', 'ignore').decode()  # Remove non-ASCII characters
#     return text

# def extract_entities_relationships(folder, prompt_template):
#     start = timer()
#     files = glob.glob(f"./data/{folder}/*")
#     system_msg = "You are a beauty and personal care expert who understands product relationships and routines."
#     print(f"Running pipeline for {len(files)} files in {folder} folder")
    
#     # First pass: collect and summarize all product information
#     product_summaries = {}  # Use dict with file_id as key for better tracking
#     summary_prompt = """
#     Create a concise summary of the product. Return ONLY valid JSON without any explanatory text:
#     {
#         "product_id": "KEEP_ORIGINAL_ID",  # Do not modify the original product ID
#         "name": "product name",
#         "brand": "brand name",
#         "category": "main category",
#         "price": float,
#         "rating": float,
#         "key_features": ["feature1", "feature2", "feature3"],
#         "main_ingredients": ["ingredient1", "ingredient2"],
#         "target_concerns": ["concern1", "concern2"],
#         "usage": "brief usage description"
#     }

#     Note: Maintain the exact product ID from the input file.
#     """
    
#     print("First pass: Collecting product summaries...")
#     for file in files:
#         try:
#             file_id = file.split("/")[-1].replace(".md", "")
#             with open(file, "r", encoding='utf-8') as f:
#                 text = clean_text(f.read().rstrip())
#                 prompt = Template(summary_prompt).substitute(ctext=text)
#                 result = process_gpt(prompt, system_msg)
#                 summary = json.loads(result)
#                 summary['file_id'] = file_id  # Ensure we keep the original file ID
#                 product_summaries[file_id] = summary
#                 print(f"Summarized {file_id}")
#         except Exception as e:
#             print(f"Error summarizing {file}: {e}")
#             continue
    
#     # Create a structured context string with product summaries
#     def create_focused_context(current_id, all_summaries, max_related=10):
#         """Create context focusing on relevant products"""
#         current_product = all_summaries[current_id]
#         other_products = []
        
#         for pid, summary in all_summaries.items():
#             if pid == current_id:
#                 continue
                
#             # Calculate relevance score
#             relevance = 0
#             if abs(summary['price'] - current_product['price']) <= 10:
#                 relevance += 1
#             if summary['brand'].lower() == current_product['brand'].lower():
#                 relevance += 2
#             if summary['category'].lower() == current_product['category'].lower():
#                 relevance += 2
            
#             other_products.append((relevance, summary))
        
#         # Sort by relevance and take top N
#         other_products.sort(reverse=True, key=lambda x: x[0])
#         related_products = other_products[:max_related]
        
#         # Create context string
#         context = "Related products for consideration:\n"
#         for _, prod in related_products:
#             context += f"""
# ID: {prod['file_id']}  # Original product ID
# Name: {clean_text(prod['name'])}
# Brand: {clean_text(prod['brand'])}
# Category: {clean_text(prod['category'])}
# Price: ${prod['price']}
# Rating: {prod['rating']}
# Key Features: {', '.join(clean_text(f) for f in prod['key_features'])}
# Usage: {clean_text(prod['usage'])}
# ---"""
        
#         return context
    
#     # Second pass: process each product with context
#     all_entities = {}
#     all_relationships = set()
    
#     print("\nSecond pass: Processing products with context...")
#     for file in files:
#         try:
#             file_id = file.split("/")[-1].replace(".md", "")
#             with open(file, "r", encoding='utf-8') as f:
#                 text = clean_text(f.read().rstrip())
                
#                 # Create focused context
#                 focused_context = create_focused_context(file_id, product_summaries)
                
#                 # Modify prompt to ensure consistent product IDs
#                 relationship_prompt = f"""
#                 Analyze the current product and create relationships with other products.
#                 IMPORTANT: 
#                 - Use EXACT product IDs as provided in the context
#                 - Do not create new product IDs
#                 - Do not modify existing product IDs
#                 - Ensure all relationships reference valid product IDs from the context

#                 Current Product ID: {file_id}
#                 {focused_context}
#                 """
                
#                 full_prompt = f"{prompt_template}\n\n{relationship_prompt}\n\nProduct to analyze:\n{text}"
                
#                 result = process_gpt(full_prompt, system_msg)
#                 parsed_result = json.loads(result)
                
#                 # Validate and add entities
#                 for entity in parsed_result["entities"]:
#                     entity_id = entity["id"]
#                     if entity_id in all_entities:
#                         # Update only if it's the same product
#                         if entity.get("label") == "Product" and entity_id != file_id:
#                             continue  # Skip if it's trying to redefine another product
#                         all_entities[entity_id].update(entity)
#                     else:
#                         all_entities[entity_id] = entity
                
#                 # Validate relationships
#                 for rel in parsed_result["relationships"]:
#                     src_id, rel_type, tgt_id = rel.split("|")
#                     # Only add relationship if both products exist in our context
#                     if src_id in product_summaries and tgt_id in product_summaries:
#                         all_relationships.add(rel)
                
#                 print(f"Processed {file_id}")
                
#         except Exception as e:
#             print(f"Error processing {file}: {e}")
#             continue
    
#     final_result = {
#         "entities": list(all_entities.values()),
#         "relationships": list(all_relationships)
#     }
    
#     end = timer()
#     print(f"Pipeline completed in {end-start} seconds")
#     return [final_result]

# # Function to take folder of files and a prompt template, and return a json-object of all the entities and relationships
# def extract_entities_relationships(folder, prompt_template):
#     start = timer()
#     files = glob.glob(f"./data/{folder}/*")
#     system_msg = "You are a helpful IT-project and account management expert who extracts information from documents."
#     print(f"Running pipeline for {len(files)} files in {folder} folder")
    
#     # Dictionary to store all entities and relationships
#     all_entities = {}  # Use ID as key to avoid duplicates
#     all_relationships = set()  # Use set to avoid duplicates
    
#     for i, file in enumerate(files):
#         print(f"Extracting entities and relationships for {file}")
#         try:
#             with open(file, "r", encoding='utf-8') as f:
#                 text = f.read().rstrip()
#                 prompt = Template(prompt_template).substitute(ctext=text)
#                 result = process_gpt(prompt, system_msg=system_msg)
#                 parsed_result = json.loads(result)
                
#                 # Add entities to master dictionary, using ID as key to avoid duplicates
#                 for entity in parsed_result["entities"]:
#                     entity_id = entity["id"]
#                     if entity_id in all_entities:
#                         # Merge attributes if entity already exists
#                         all_entities[entity_id].update(entity)
#                     else:
#                         all_entities[entity_id] = entity
                
#                 # Add relationships to master set
#                 all_relationships.update(parsed_result["relationships"])
                
#         except Exception as e:
#             print(f"Error processing {file}: {e}")
    
#     # Convert back to the expected format
#     final_result = {
#         "entities": list(all_entities.values()),
#         "relationships": list(all_relationships)
#     }
    
#     end = timer()
#     print(f"Pipeline completed in {end-start} seconds")
#     return [final_result]

# Update the ingestion_pipeline function to handle the merged results
def ingestion_pipeline(folders):
    entities_relationships = []
    for key, value in folders.items():
        entities_relationships.extend(extract_entities_relationships(key, value))
    
    # Generate and execute cypher statements
    cypher_statements = generate_cypher(entities_relationships)
    for i, stmt in enumerate(cypher_statements):
        print(f"Executing cypher statement {i+1} of {len(cypher_statements)}\"")
        try:
            gds.execute_query(stmt)
        except Exception as e:
            with open("failed_statements.txt", "w") as f:
                f.write(f"{stmt} - Exception: {e}\n")


# Function to take a json-object of entitites and relationships and generate cypher query for creating those entities
# def generate_cypher(json_obj):
#     e_statements = []
#     r_statements = []

#     e_label_map = {}

#     # loop through our json object
#     for i, obj in enumerate(json_obj):
#         print(f"Generating cypher for file {i+1} of {len(json_obj)}")
#         for entity in obj["entities"]:
#             label = entity["label"]
#             id = entity["id"]
#             id = id.replace("-", "").replace("_", "")
#             properties = {k: v for k, v in entity.items() if k not in ["label", "id"]}

#             cypher = f'MERGE (n:{label} {{id: "{id}"}})'
#             if properties:
#                 props_str = ", ".join(
#                     [f'n.{key} = "{val}"' for key, val in properties.items()]
#                 )
#                 cypher += f" ON CREATE SET {props_str}"
#             e_statements.append(cypher)
#             e_label_map[id] = label

#         for rs in obj["relationships"]:
#             src_id, rs_type, tgt_id = rs.split("|")
#             src_id = src_id.replace("-", "").replace("_", "")
#             tgt_id = tgt_id.replace("-", "").replace("_", "")

#             src_label = e_label_map[src_id]
#             tgt_label = e_label_map[tgt_id]

#             cypher = f'MERGE (a:{src_label} {{id: "{src_id}"}}) MERGE (b:{tgt_label} {{id: "{tgt_id}"}}) MERGE (a)-[:{rs_type}]->(b)'
#             r_statements.append(cypher)

#     with open("cyphers.txt", "w") as outfile:
#         outfile.write("\n".join(e_statements + r_statements))

#     return e_statements + r_statements
def generate_cypher(json_obj):
    e_statements = []
    r_statements = []
    e_label_map = {}
    skipped_relationships = []

    # First pass: collect all entities
    for i, obj in enumerate(json_obj):
        print(f"Generating cypher for file {i+1} of {len(json_obj)}")
        for entity in obj["entities"]:
            label = entity["label"]
            id = entity["id"]
            id = id.replace("-", "").replace("_", "")
            properties = {k: v for k, v in entity.items() if k not in ["label", "id"]}

            cypher = f'MERGE (n:{label} {{id: "{id}"}})'
            if properties:
                props_str = ", ".join(
                    [f'n.{key} = "{val}"' for key, val in properties.items()]
                )
                cypher += f" ON CREATE SET {props_str}"
            e_statements.append(cypher)
            e_label_map[id] = label

    # Second pass: create relationships only between existing entities
    for i, obj in enumerate(json_obj):
        for rs in obj["relationships"]:
            try:
                src_id, rs_type, tgt_id = rs.split("|")
                src_id = src_id.replace("-", "").replace("_", "")
                tgt_id = tgt_id.replace("-", "").replace("_", "")

                # Check if both source and target entities exist
                if src_id in e_label_map and tgt_id in e_label_map:
                    src_label = e_label_map[src_id]
                    tgt_label = e_label_map[tgt_id]

                    cypher = f'MERGE (a:{src_label} {{id: "{src_id}"}}) MERGE (b:{tgt_label} {{id: "{tgt_id}"}}) MERGE (a)-[:{rs_type}]->(b)'
                    r_statements.append(cypher)
                else:
                    skipped_relationships.append(rs)
            except Exception as e:
                print(f"Error processing relationship {rs}: {e}")
                skipped_relationships.append(rs)

    # Log skipped relationships
    if skipped_relationships:
        print("\nWarning: The following relationships were skipped due to missing entities:")
        for rs in skipped_relationships:
            print(f"- {rs}")
        
        with open("skipped_relationships.txt", "w") as f:
            f.write("\n".join(skipped_relationships))
            print("\nSkipped relationships have been written to 'skipped_relationships.txt'")

    # Write all valid statements to file
    with open("cyphers.txt", "w") as outfile:
        outfile.write("\n".join(e_statements + r_statements))

    return e_statements + r_statements


# Final function to bring all the steps together
def ingestion_pipeline(folders):
    # Extrating the entites and relationships from each folder, append into one json_object
    entities_relationships = []
    for key, value in folders.items():
        entities_relationships.extend(extract_entities_relationships(key, value))

    # Generate and execute cypher statements
    cypher_statements = generate_cypher(entities_relationships)
    for i, stmt in enumerate(cypher_statements):
        print(f"Executing cypher statement {i+1} of {len(cypher_statements)}")
        try:
            gds.execute_query(stmt)
        except Exception as e:
            with open("failed_statements.txt", "w") as f:
                f.write(f"{stmt} - Exception: {e}\n")

### 3. Defining Prompts

In [101]:
amazon_products_prompt_template = """
From the product description below, extract Entities & Relationships in an open format. You are free to:
1. Create any entity types you identify, with any relevant attributes. Each entity must have at least:
    - label: The type of entity (e.g., 'Product', 'Ingredient', 'Benefit', etc.)
    - id: A unique alphanumeric identifier
    - Additional attributes specific to that entity type

2. Create any relationships you can identify:
    - Between entities within this product description
        - SAME_BRAND_AS: Products from the same manufacturer
        - SIMILAR_PRICE_TIER_AS: Products within 5 dollar price range
        - SIMILAR_RATING_AS: Products with similar ratings (within 0.5 rating score)
        - USED_WITH: Products commonly used together in a routine
        - ALTERNATIVE_TO: Similar products that serve the same purpose
        - COMPLEMENTS: Products that enhance each other's effects
        - PART_OF_COLLECTION: Products in the same product line
        - SHARES_INGREDIENT_WITH: Products with common key ingredients
        - ADDRESSES_SAME_CONCERN: Products targeting the same issues
        - USED_IN_SAME_STEP: Products used at the same routine step
    - With potential related products (use consistent IDs for products you reference)
    - With any beauty/health concepts, ingredients, or categories

3. The output MUST be valid JSON and should be formatted as:
{
    "entities": [
        {"label":"string","id":"string",...additional_attributes},
        ...
    ],
    "relationships": [
        "sourceid|RELATIONSHIP_TYPE|targetid",
        ...
    ]
}

Important: When referencing other products or creating relationships across products:
- Use predictable, consistent IDs for well-known products or concepts
- If you reference another product, include it as an entity with as much information as you can infer
- Create meaningful relationship types that explain how products connect

Case Sheet:
$ctext
"""

# amazon_products_prompt_template = """
# From the product description below, extract Entities & Relationships in an open format, ensuring connections to other beauty/personal care products. Consider:

# 1. Entity Types (include but not limited to):
#     - Products (with price, rating, brand, type)
#     - Brands/Manufacturers
#     - Product Categories and Subcategories
#     - Price Ranges (0 to 15 dollars: Budget, 15 to 30 dollars: Mid-range, 30 or more dollars: Premium)
#     - Rating Tiers (4.5 to 5: Excellent, 4 to 4.4: Good, 3 to 3.9: Average, less than 3: Poor)
#     - Usage Occasions (Morning, Night, Special Event, etc.)
#     - Target Concerns (Aging, Acne, Dryness, etc.)
#     - Key Ingredients
#     - Product Functions (Cleansing, Moisturizing, Protection, etc.)

# 2. Cross-Product Relationships:
#     - SAME_BRAND_AS: Products from the same manufacturer
#     - SIMILAR_PRICE_TIER_AS: Products in the same price range
#     - SIMILAR_RATING_AS: Products with similar ratings (within 0.5 points)
#     - USED_WITH: Products commonly used together in a routine
#     - ALTERNATIVE_TO: Similar products that serve the same purpose
#     - COMPLEMENTS: Products that enhance each other's effects
#     - PART_OF_COLLECTION: Products in the same product line
#     - SHARES_INGREDIENT_WITH: Products with common key ingredients
#     - ADDRESSES_SAME_CONCERN: Products targeting the same issues
#     - USED_IN_SAME_STEP: Products used at the same routine step

# 3. The output MUST be valid JSON and should be formatted as:
# {
#     "entities": [
#         {
#             "label": "string",
#             "id": "string",
#             "name": "string",
#             "price": float,
#             "rating": float,
#             "brand": "string",
#             "category": "string",
#             ... (additional attributes as needed)
#         }
#     ],
#     "relationships": [
#         "sourceid|RELATIONSHIP_TYPE|targetid"
#     ]
# }

# Important Notes:
# - Create relationships with other likely related products even if not explicitly mentioned
# - Use consistent IDs for brands, categories, and common product types
# - Consider price points and ratings when suggesting relationships
# - Think about typical beauty/personal care routines when creating USED_WITH relationships
# - Include relationships to generic product categories (e.g., "moisturizer", "cleanser")
# - Consider seasonal or occasion-based relationships
# - Look for complementary products in typical beauty routines

# Case Sheet:
# $ctext
# """

### 4. Running the pipeline

In [102]:
folders = {
    "amazon_products": amazon_products_prompt_template,
}
ingestion_pipeline(folders)

Running pipeline for 5 files in amazon_products folder
First pass: Collecting product summaries...
Summarized unstructured2
Summarized unstructured3
Summarized unstructured4
Summarized unstructured5
Summarized unstructured1

Second pass: Processing products with context...
Processed unstructured2
Processed unstructured3
Processed unstructured4
Processed unstructured5
Processed unstructured1
Pipeline completed in 223.90520516596735 seconds
Generating cypher for file 1 of 1
Executing cypher statement 1 of 5
Executing cypher statement 2 of 5
Executing cypher statement 3 of 5
Executing cypher statement 4 of 5
Executing cypher statement 5 of 5


In [47]:
def execute_cypher_from_file(filepath):
    """
    Reads Cypher queries from a text file and executes them sequentially.
    
    Args:
        filepath (str): Path to the text file containing Cypher queries (one per line)
    """
    try:
        # Read the cypher statements from file
        with open(filepath, 'r') as file:
            cypher_statements = [line.strip() for line in file if line.strip()]
        
        print(f"Found {len(cypher_statements)} Cypher statements to execute")
        
        # Execute each statement
        failed_statements = []
        for i, stmt in enumerate(cypher_statements, 1):
            try:
                print(f"Executing statement {i} of {len(cypher_statements)}")
                gds.execute_query(stmt)
            except Exception as e:
                print(f"Error executing statement {i}: {e}")
                failed_statements.append((stmt, str(e)))
        
        # Report results
        success_count = len(cypher_statements) - len(failed_statements)
        print(f"\nExecution complete:")
        print(f"Successfully executed: {success_count}/{len(cypher_statements)} statements")
        
        # If there were any failures, write them to a file
        if failed_statements:
            print(f"Failed statements: {len(failed_statements)}")
            with open("failed_statements.txt", "w") as f:
                for stmt, error in failed_statements:
                    f.write(f"Statement: {stmt}\nError: {error}\n\n")
            print("Failed statements have been written to 'failed_statements.txt'")
            
    except FileNotFoundError:
        print(f"Error: Could not find file '{filepath}'")
    except Exception as e:
        print(f"Unexpected error: {e}")
    finally:
        # Ensure the database connection is closed properly
        gds.close()

In [49]:
execute_cypher_from_file("cyphers.txt")

Found 64 Cypher statements to execute
Executing statement 1 of 64


/var/folders/cf/2y7jfcdj6dbfw7ljc4s4jpv40000gn/T/ipykernel_36087/4009459450.py:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  gds.execute_query(stmt)


Executing statement 2 of 64
Executing statement 3 of 64
Executing statement 4 of 64
Executing statement 5 of 64
Executing statement 6 of 64
Executing statement 7 of 64
Executing statement 8 of 64
Executing statement 9 of 64
Executing statement 10 of 64
Executing statement 11 of 64
Executing statement 12 of 64
Executing statement 13 of 64
Executing statement 14 of 64
Executing statement 15 of 64
Executing statement 16 of 64
Executing statement 17 of 64
Executing statement 18 of 64
Executing statement 19 of 64
Executing statement 20 of 64
Executing statement 21 of 64
Executing statement 22 of 64
Executing statement 23 of 64
Executing statement 24 of 64
Executing statement 25 of 64
Executing statement 26 of 64
Executing statement 27 of 64
Executing statement 28 of 64
Executing statement 29 of 64
Executing statement 30 of 64
Executing statement 31 of 64
Executing statement 32 of 64
Executing statement 33 of 64
Executing statement 34 of 64
Executing statement 35 of 64
Executing statement 36